In [1]:
# pip install pdfkit requests

In [6]:
# pip install frontend

     ---------------------------------------- 71.9/71.9 kB 3.9 MB/s eta 0:00:00
     ---------------------------------------- 62.4/62.4 kB 3.5 MB/s eta 0:00:00
     ---------------------------------------- 58.3/58.3 kB ? eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install pdfkit requests PyPDF2 beautifulsoup4 lxml pymupdf fitz

     ---------------------------------------- 3.2/3.2 MB 9.7 MB/s eta 0:00:00
     ---------------------------------------- 96.9/96.9 kB 5.8 MB/s eta 0:00:00
     ---------------------------------------- 3.3/3.3 MB 13.1 MB/s eta 0:00:00
     ---------------------------------------- 95.6/95.6 kB 5.3 MB/s eta 0:00:00
     ---------------------------------------- 5.0/5.0 MB 10.0 MB/s eta 0:00:00
     ---------------------------------------- 75.5/75.5 kB 4.1 MB/s eta 0:00:00
     ------------------------------------- 531.9/531.9 kB 11.3 MB/s eta 0:00:00
     ------------------------------------- 421.5/421.5 kB 13.3 MB/s eta 0:00:00
     ------------------------------------- 528.1/528.1 kB 11.0 MB/s eta 0:00:00
     ---------------------------------------- 41.7/41.7 kB ? eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
# pip install wkhtmltopdf

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for wkhtmltopdf: filename=wkhtmltopdf-0.2-py3-none-any.whl size=11133 sha256=0a4e5fcb15c789fefea97fe6961d9cb440f17e13ad974ad2ffe087169a7a59e4
  Stored in directory: c:\users\chris\appdata\local\pip\cache\wheels\d9\ac\1d\25b42a14d8813a8f1c2420ce552e84c547a28a9631dfdfad39
Successfully built wkhtmltopdf
Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import pdfkit
import requests
from PyPDF2 import PdfReader, PdfMerger
import re

MAX_FILENAME_LENGTH = 50  # Maximum length for filenames

def download_pdf(url, output_file):
    """
    Download a PDF file from a given URL.

    Parameters:
    url (str): URL of the PDF file.
    output_file (str): The output file path.
    """
    response = requests.get(url)
    with open(output_file, 'wb') as file:
        file.write(response.content)

def is_pdf(url):
    """
    Check if the given URL points to a PDF.

    Parameters:
    url (str): URL to check.

    Returns:
    bool: True if the URL points to a PDF, False otherwise.
    """
    response = requests.head(url, allow_redirects=True)
    return 'application/pdf' in response.headers.get('Content-Type', '')

def sanitize_filename(filename):
    """
    Sanitize a filename by removing invalid characters and truncating it to a maximum length.

    Parameters:
    filename (str): The filename to sanitize.

    Returns:
    str: The sanitized filename.
    """
    sanitized = re.sub(r'[\\/*?:"<>|]', "", filename.strip())
    return sanitized[:MAX_FILENAME_LENGTH]

def get_filename_from_url(url):
    """
    Generate a filename from the URL.

    Parameters:
    url (str): The URL to generate a filename from.

    Returns:
    str: The generated filename.
    """
    return sanitize_filename(url.split('//')[-1].replace('/', '_'))

def get_title_from_pdf(pdf_path):
    """
    Extract the first line or title from a PDF file.

    Parameters:
    pdf_path (str): Path to the PDF file.

    Returns:
    str: The extracted title or a sanitized filename part as a fallback.
    """
    reader = PdfReader(pdf_path)
    if len(reader.pages) > 0:
        first_page = reader.pages[0]
        text = first_page.extract_text().split('\n')[0]
        truncated_text = text[:MAX_FILENAME_LENGTH].strip()
        return sanitize_filename(truncated_text)
    return os.path.splitext(os.path.basename(pdf_path))[0]

def generate_unique_filename(filename, existing_files):
    """
    Generate a unique filename by appending a number if the filename already exists.

    Parameters:
    filename (str): The desired filename.
    existing_files (set): A set of filenames that already exist.

    Returns:
    str: A unique filename.
    """
    base, ext = os.path.splitext(filename)
    counter = 1
    unique_filename = filename
    while unique_filename in existing_files:
        unique_filename = f"{base}_{counter}{ext}"
        counter += 1
    existing_files.add(unique_filename)
    return unique_filename

def urls_to_pdf(urls, output_file):
    """
    Convert a list of URLs to a single PDF file. Handles both direct PDF links and webpages.

    Parameters:
    urls (list): List of URLs to be converted.
    output_file (str): The output PDF file name.
    """
    pdf_files = []
    existing_files = set()
    path_to_wkhtmltopdf = r'C:\Program Files\wkhtmltopdf\bin\wkhtmltopdf.exe'  # Use the correct path
    config = pdfkit.configuration(wkhtmltopdf=path_to_wkhtmltopdf)

    options = {
        'quiet': '',
        'no-stop-slow-scripts': '',
        'disable-smart-shrinking': '',
        'javascript-delay': '5000'  # Increase delay to 5 seconds
    }

    for url in urls:
        if is_pdf(url):
            # Website provided is PDF
            temp_pdf = "temp.pdf"
            download_pdf(url, temp_pdf)
            title = get_title_from_pdf(temp_pdf)
            pdf_output = f"{title}.pdf"
            pdf_output = generate_unique_filename(pdf_output, existing_files)
            try:
                os.rename(temp_pdf, pdf_output)
            except FileNotFoundError:
                pass
            except OSError as e:
                print(f"OSError renaming {temp_pdf} to {pdf_output}: {e}")
            pdf_files.append(pdf_output)
        else:
            # Website provided was URL
            temp_pdf = "output.pdf"

            # Generate PDF
            pdfkit.from_url(url, temp_pdf, configuration=config, options=options)

            # Get unique name
            filename = get_filename_from_url(url)
            pdf_output = f"{filename}.pdf"
            
            # Check duplications
            pdf_output = generate_unique_filename(pdf_output, existing_files)

            # Rename file:
            try:
                os.rename(temp_pdf, pdf_output)
            except FileNotFoundError:
                pass
            except OSError as e:
                print(f"OSError renaming {temp_pdf} to {pdf_output}: {e}")

            # Track List:
            pdf_files.append(pdf_output)

        print(pdf_files)

    # # Merge all PDFs into a single PDF
    # merger = PdfMerger()

    # for pdf in pdf_files:
    #     merger.append(pdf)

    # merger.write(output_file)  # Save the merged PDF with the specified output filename
    # merger.close()

    # # Clean up individual PDFs
    # for pdf in pdf_files:
    #     try:
    #         os.remove(pdf)
    #     except FileNotFoundError:
    #         pass
    #     except OSError as e:
    #         print(f"OSError removing {pdf}: {e}")



# Example usage
urls = ['https://cloud.google.com/blog/transform/how-to-be-a-better-prompt-engineer',
        # 'https://arxiv.org/pdf/2311.05661',
        # 'https://arxiv.org/pdf/2311.16452',
        # 'https://arxiv.org/abs/2306.12509',
        # 'https://arxiv.org/pdf/2401.14423',
]

output_file = 'output.pdf'
urls_to_pdf(urls, output_file)


['cloud.google.com_blog_transform_how-to-be-a-better.pdf']


In [ ]:
# Blocks of Code not required:
    # # Merge all PDFs into a single PDF
    # merger = PdfMerger()

    # for pdf in pdf_files:
    #     merger.append(pdf)

    # merger.write(output_file)  # Save the merged PDF with the specified output filename
    # merger.close()

    # # Clean up individual PDFs
    # for pdf in pdf_files:
    #     os.remove(pdf)

    # Debugging information
    # print(f"Converting URL to PDF: {url}")
    # print(f"Saving as: {pdf_output}")

            # # Rename file. Ignore the filenotfounderror, since the renaming still happens.
            # try:
            #     os.rename("output.pdf", pdf_output)
            # except Exception:
            #     pass

# Example usage
# urls = ['https://digitalcommons.chapman.edu/cgi/viewcontent.cgi?article=1044&context=librarian_articles',
# 'https://arxiv.org/pdf/2311.05232',
# 'https://arxiv.org/pdf/2312.16171v1',
# 'https://arxiv.org/pdf/2310.05029',
# 'https://arxiv.org/pdf/2310.01714',
# 'https://arxiv.org/pdf/2309.11495',
# 'https://arxiv.org/pdf/2309.04269',
# 'https://arxiv.org/pdf/2310.08395',
# 'https://arxiv.org/pdf/2310.08123',
# 'https://arxiv.org/pdf/2310.08101',
# 'https://arxiv.org/pdf/2310.00297'
# ]